In [59]:
import warnings
import pandas as pd
import numpy as np
import glob
import os
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import dump
warnings.filterwarnings('ignore')

# Set data paths
flight_data_path = './cleaned_data/'
weather_data_path = './cleaned_weather_data/'
top_airports_file = './top_100_airports.csv'
output_dir = './cancelled_prob_rf_models/'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(os.path.join(output_dir, 'metrics'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'plots'), exist_ok=True)

print("Starting flight cancellation prediction models with red-eye flight detection (Random Forest)...")
print(f"Flight data directory: {flight_data_path}")
print(f"Weather data directory: {weather_data_path}")
print(f"Top airports file: {top_airports_file}")
print(f"Model output directory: {output_dir}")

# Load top 30 airports from the top 100 airports file
try:
    top_airports = pd.read_csv(top_airports_file, low_memory=False)

    top_airports = top_airports.head(30)

    top_airport_codes = set(top_airports['ORIGIN_IATA'].str.strip().tolist())

    print(f"Loaded top 30 airports: {', '.join(sorted(top_airport_codes))}")
    print(f"Busiest airport: {top_airports.iloc[0]['ORIGIN_IATA']} with {top_airports.iloc[0]['Times']} flights")
    print(f"30th busiest airport: {top_airports.iloc[29]['ORIGIN_IATA']} with {top_airports.iloc[29]['Times']} flights")
except Exception as e:
    print(f"Error loading top airports file: {e}")
    top_airport_codes = None
    print("Will process all airports (top airports file not available)")

Starting flight cancellation prediction models with red-eye flight detection (Random Forest)...
Flight data directory: ./cleaned_data/
Weather data directory: ./cleaned_weather_data/
Top airports file: ./top_100_airports.csv
Model output directory: ./cancelled_prob_rf_models/
Loaded top 30 airports: ATL, AUS, BNA, BOS, BWI, CLT, DCA, DEN, DFW, DTW, EWR, FLL, IAD, IAH, JFK, LAS, LAX, LGA, MCO, MDW, MIA, MSP, ORD, PHL, PHX, SAN, SEA, SFO, SLC, TPA
Busiest airport: ATL with 457121 flights
30th busiest airport: TPA with 97235 flights


In [60]:
def load_weather_data():
    print("\nLoading weather data...")
    start_time = time.time()

    all_files = glob.glob(os.path.join(weather_data_path, "*.csv"))
    print(f"Found {len(all_files)} total weather data files")
    weather_dict = {}
    count = 0
    matching_count = 0

    # Process all weather files
    for file in all_files:
        try:
            filename = os.path.basename(file)
            parts = filename.split('.')[0].split('_')

            if len(parts) >= 3:
                iata = parts[0]
                year = parts[1]
                month_name = parts[2]

                month_map = {
                    'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04',
                    'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08',
                    'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'
                }

                if month_name in month_map:
                    month = month_map[month_name]

                    if top_airport_codes is None or iata in top_airport_codes:
                        weather_data = pd.read_csv(file, low_memory=False)

                        if 'DATE' not in weather_data.columns:
                            print(f"Warning: DATE column not found in {filename}")
                            continue

                        weather_data['DATE'] = pd.to_datetime(weather_data['DATE'])

                        key = f"{iata}_{year}_{month}"

                        # Store the weather data
                        weather_dict[key] = weather_data
                        matching_count += 1
                else:
                    print(f"Warning: Unknown month format in {filename}")

                count += 1

                # Print progress periodically
                if count % 100 == 0:
                    print(f"Processed {count} weather files, loaded {matching_count} matching files")
        except Exception as e:
            print(f"Error loading weather file {file}: {e}")

    print(f"Loaded {matching_count} weather files out of {count} processed files")
    print(f"Loading weather data took: {time.time() - start_time:.2f} seconds")
    return weather_dict

# Function to match weather data to flights for origin airports (departure weather)
def match_weather_data(df):
    print("\nMatching origin weather data with flights...")
    start_time = time.time()

    date_columns_exist = all(col in df.columns for col in ['YEAR', 'MONTH', 'DAY'])
    if not date_columns_exist:
        print("Warning: Missing one or more date columns (YEAR, MONTH, DAY)")
        print("Weather data cannot be matched")
        return df

    df['FLIGHT_DATE'] = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']])
    df['WEATHER_KEY'] = df['ORIGIN_IATA'] + '_' + df['YEAR'].astype(str) + '_' + df['MONTH'].astype(str).str.zfill(2)
    weather_columns = ['EXTREME_WEATHER', 'PRCP', 'WT01', 'WT03', 'WT04', 'WT05', 'WT08', 'WT11']
    for col in weather_columns:
        if col not in df.columns:
            df[col] = 0.0

    # Process in batches
    matched_count = 0
    batch_size = 10000

    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))
        batch = df.iloc[start_idx:end_idx]

        for idx, row in batch.iterrows():
            try:
                weather_key = row['WEATHER_KEY']
                flight_date = row['FLIGHT_DATE']

                if weather_key in weather_dict:
                    weather_data = weather_dict[weather_key]

                    matching_weather = weather_data[weather_data['DATE'] == flight_date]

                    if not matching_weather.empty:
                        for col in weather_columns:
                            if col in matching_weather.columns:
                                df.at[idx, col] = matching_weather[col].iloc[0]
                        matched_count += 1
            except Exception as e:
                pass

        # Print progress
        print(f"Processed {end_idx}/{len(df)} rows, matched {matched_count} flights with origin weather data")

    print(f"Matched origin weather data for {matched_count} flights ({matched_count/len(df)*100:.2f}%)")
    print(f"Origin weather matching took: {time.time() - start_time:.2f} seconds")

    return df

In [61]:
# Function Match destination weather data
def match_destination_weather_data(df):
    print("\nMatching destination weather data with flights...")
    start_time = time.time()

    date_columns_exist = all(col in df.columns for col in ['YEAR', 'MONTH', 'DAY'])
    if not date_columns_exist:
        print("Warning: Missing one or more date columns (YEAR, MONTH, DAY)")
        print("Destination weather data cannot be matched")
        return df

    if 'FLIGHT_DATE' not in df.columns:
        df['FLIGHT_DATE'] = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']])

    df['DEST_WEATHER_KEY'] = df['DEST_IATA'] + '_' + df['YEAR'].astype(str) + '_' + df['MONTH'].astype(str).str.zfill(2)

    # Create columns for destination weather features with DEST_ prefix
    weather_columns = ['EXTREME_WEATHER', 'PRCP', 'WT01', 'WT03', 'WT04', 'WT05', 'WT08', 'WT11']
    for col in weather_columns:
        dest_col = f'DEST_{col}'
        if dest_col not in df.columns:
            df[dest_col] = 0.0

    # Process in batches
    matched_count = 0
    batch_size = 10000

    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))
        batch = df.iloc[start_idx:end_idx]

        for idx, row in batch.iterrows():
            try:
                dest_weather_key = row['DEST_WEATHER_KEY']
                flight_date = row['FLIGHT_DATE']

                if dest_weather_key in weather_dict:
                    weather_data = weather_dict[dest_weather_key]

                    matching_weather = weather_data[weather_data['DATE'] == flight_date]

                    if not matching_weather.empty:
                        for col in weather_columns:
                            if col in matching_weather.columns:
                                df.at[idx, f'DEST_{col}'] = matching_weather[col].iloc[0]
                        matched_count += 1
            except Exception as e:
                pass

        print(f"Processed {end_idx}/{len(df)} rows, matched {matched_count} flights with destination weather data")

    print(f"Matched destination weather data for {matched_count} flights ({matched_count/len(df)*100:.2f}%)")
    print(f"Destination weather matching took: {time.time() - start_time:.2f} seconds")

    return df

# Get specific May files from the cleaned_data directory based on the file list you shared
def get_may_files():
    may_files = [
        os.path.join(flight_data_path, "May2021.csv"),
        os.path.join(flight_data_path, "May2022.csv"),
        os.path.join(flight_data_path, "May2023.csv"),
        os.path.join(flight_data_path, "May2024.csv")
    ]

    existing_files = []
    for file_path in may_files:
        if os.path.exists(file_path):
            existing_files.append(file_path)
        else:
            print(f"Warning: File {file_path} not found")

    return existing_files

# Get the May 2021-2024 flight data files
flight_files = get_may_files()
print(f"\nFound {len(flight_files)} May files to process:")
for f in flight_files:
    print(f"  - {os.path.basename(f)}")

if not flight_files:
    print("No May 2021-2024 files were found. Please check file paths.")
    exit(1)

# Load all weather data once (shared across all models)
weather_dict = load_weather_data()

# Function to extract year from filename (for logging purposes only)
def extract_year_from_filename(filename):
    base_name = os.path.basename(filename)
    year_str = base_name.replace('May', '').split('.')[0]
    return int(year_str)

# Function to create red-eye flight indicator
def create_redeye_indicator(df):
    df = df.copy()

    df['IS_REDEYE'] = 0

    dep_time_cols = [col for col in df.columns if 'DEP_TIME' in col.upper()]

    arr_time_cols = [col for col in df.columns if 'ARR_TIME' in col.upper()]

    if dep_time_cols:
        dep_time_col = dep_time_cols[0]

        if df[dep_time_col].dtype != 'float64':
            try:
                df[dep_time_col] = df[dep_time_col].astype(float)
            except:

                print(f"Warning: Could not convert {dep_time_col} to float")

        redeye_departure = (df[dep_time_col] >= 0) & (df[dep_time_col] < 600)
        df.loc[redeye_departure, 'IS_REDEYE'] = 1

    # If we have arrival time
    if arr_time_cols:
        arr_time_col = arr_time_cols[0]

        if df[arr_time_col].dtype != 'float64':
            try:
                df[arr_time_col] = df[arr_time_col].astype(float)
            except:
                print(f"Warning: Could not convert {arr_time_col} to float")

        redeye_arrival = (df[arr_time_col] >= 0) & (df[arr_time_col] < 600)
        df.loc[redeye_arrival, 'IS_REDEYE'] = 1

    redeye_count = df['IS_REDEYE'].sum()
    total_count = len(df)
    print(f"Identified {redeye_count} red-eye flights out of {total_count} total flights ({redeye_count/total_count*100:.2f}%)")

    return df

# Function to create additional indicator variables
def create_indicator_variables(df):
    df = df.copy()

    df['IS_WEEKEND'] = 0
    df['IS_MORNING_PEAK'] = 0
    df['IS_EVENING_PEAK'] = 0

    if 'WEEK' in df.columns:
        weekend_mask = (df['WEEK'] == 0) | (df['WEEK'] == 6)
        df.loc[weekend_mask, 'IS_WEEKEND'] = 1

    dep_time_cols = [col for col in df.columns if 'DEP_TIME' in col.upper()]

    if dep_time_cols:
        dep_time_col = dep_time_cols[0]
        if df[dep_time_col].dtype != 'float64':
            try:
                df[dep_time_col] = df[dep_time_col].astype(float)
            except:
                print(f"Warning: Could not convert {dep_time_col} to float")

        # Morning peak hours: 7:00 to 10:00 AM (700-1000)
        morning_peak = (df[dep_time_col] >= 700) & (df[dep_time_col] < 1000)
        df.loc[morning_peak, 'IS_MORNING_PEAK'] = 1

        # Evening peak hours: 4:00 to 7:00 PM (1600-1900)
        evening_peak = (df[dep_time_col] >= 1600) & (df[dep_time_col] < 1900)
        df.loc[evening_peak, 'IS_EVENING_PEAK'] = 1

    weekend_count = df['IS_WEEKEND'].sum()
    morning_peak_count = df['IS_MORNING_PEAK'].sum()
    evening_peak_count = df['IS_EVENING_PEAK'].sum()
    total_count = len(df)

    print(f"Identified {weekend_count} weekend flights ({weekend_count/total_count*100:.2f}%)")
    print(f"Identified {morning_peak_count} morning peak flights ({morning_peak_count/total_count*100:.2f}%)")
    print(f"Identified {evening_peak_count} evening peak flights ({evening_peak_count/total_count*100:.2f}%)")

    return df

# Function to convert day of week to correct format: Sun=0, Mon=1, Tue=2, etc.
def standardize_day_of_week(df):
    df = df.copy()

    # Ensure WEEK column exists
    if 'WEEK' not in df.columns:
        if 'DAY_OF_WEEK' in df.columns:
            df['WEEK'] = df['DAY_OF_WEEK']
        elif 'DAY' in df.columns and 'MONTH' in df.columns and 'YEAR' in df.columns:
            if 'DATE' not in df.columns:
                df['DATE'] = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']])
            df['WEEK'] = (df['DATE'].dt.dayofweek + 1) % 7
        else:
            print("Cannot create WEEK column. Required columns missing.")
            return df


    if df['WEEK'].dtype == 'object':
        day_map = {
            'Sun': 0, 'Sunday': 0,
            'Mon': 1, 'Monday': 1,
            'Tue': 2, 'Tuesday': 2,
            'Wed': 3, 'Wednesday': 3,
            'Thu': 4, 'Thursday': 4,
            'Fri': 5, 'Friday': 5,
            'Sat': 6, 'Saturday': 6
        }
        df['WEEK'] = df['WEEK'].map(day_map)

    # Convert to integer type
    df['WEEK'] = df['WEEK'].astype(int)

    # Verify the range of values is correct (0-6)
    min_val = df['WEEK'].min()
    max_val = df['WEEK'].max()

    print(f"Day of week range: {min_val} to {max_val} (0=Sunday, 1=Monday, ..., 6=Saturday)")

    if min_val < 0 or max_val > 6:
        print("Warning: Day of week values outside expected range (0-6)")

    return df


Found 4 May files to process:
  - May2021.csv
  - May2022.csv
  - May2023.csv
  - May2024.csv

Loading weather data...
Found 3550 total weather data files
Processed 100 weather files, loaded 0 matching files
Processed 200 weather files, loaded 0 matching files
Processed 300 weather files, loaded 16 matching files
Processed 400 weather files, loaded 32 matching files
Processed 500 weather files, loaded 32 matching files
Processed 600 weather files, loaded 48 matching files
Processed 700 weather files, loaded 48 matching files
Processed 800 weather files, loaded 48 matching files
Processed 900 weather files, loaded 64 matching files
Processed 1000 weather files, loaded 64 matching files
Processed 1100 weather files, loaded 80 matching files
Processed 1200 weather files, loaded 96 matching files
Processed 1300 weather files, loaded 112 matching files
Processed 1400 weather files, loaded 112 matching files
Processed 1500 weather files, loaded 112 matching files
Processed 1600 weather file

In [62]:
# Function to train a Random Forest model for a single CSV file
def train_model_for_file(file_path, file_index, total_files):
    file_name = os.path.basename(file_path)
    model_name = os.path.splitext(file_name)[0]
    file_year = extract_year_from_filename(file_name)

    print(f"\nProcessing file {file_index+1}/{total_files}: {file_name} (May {file_year})")
    start_time = time.time()

    # Load flight data from this file
    try:
        flight_df = pd.read_csv(file_path, low_memory=False)
        original_size = len(flight_df)

        if 'MONTH' in flight_df.columns:
            month_counts = flight_df['MONTH'].value_counts()
            print(f"Months found in data: {dict(month_counts)}")

            if 5 in month_counts:
                flight_df = flight_df[flight_df['MONTH'] == 5]
                print(f"Filtered to only May data: {len(flight_df)} rows")
            else:
                print(f"Warning: No May data found in file, but proceeding anyway as this should be May data based on filename")

        if top_airport_codes is not None:
            flight_df = flight_df[
                flight_df['ORIGIN_IATA'].str.strip().isin(top_airport_codes) &
                flight_df['DEST_IATA'].str.strip().isin(top_airport_codes)
            ]

            filtered_size = len(flight_df)
            print(f"Filtered from {original_size} to {filtered_size} rows for top 30 airports")

            # If no data left after filtering, skip this file
            if filtered_size == 0:
                print(f"No data remaining after filtering for top 30 airports. Skipping file.")
                return {
                    'file_name': file_name,
                    'status': 'skipped',
                    'reason': 'empty_after_filtering'
                }
    except Exception as e:
        print(f"Error loading flight data file {file_path}: {e}")
        return {
            'file_name': file_name,
            'status': 'error',
            'reason': str(e)
        }

    #print("Standardizing day of week...")
    flight_df = standardize_day_of_week(flight_df)

    #print("Creating red-eye flight indicator...")
    flight_df = create_redeye_indicator(flight_df)

    #print("Creating additional indicators: IS_WEEKEND, IS_MORNING_PEAK, IS_EVENING_PEAK...")
    flight_df = create_indicator_variables(flight_df)

    # Basic information
    print(f"Final dataset shape: {flight_df.shape}")

    print("Preprocessing data...")

    # Create target variable
    if 'CANCELLED' in flight_df.columns:
        flight_df['IS_CANCELLED'] = flight_df['CANCELLED'].astype(int)
    else:
        print("No CANCELLED column found. Skipping file.")
        return {
            'file_name': file_name,
            'status': 'skipped',
            'reason': 'no_cancelled_column'
        }

    cancelled_count = flight_df['IS_CANCELLED'].sum()
    total_count = len(flight_df)

    # Skip if there are no cancellations (can't train a model)
    if cancelled_count == 0:
        print(f"No cancelled flights in this dataset. Skipping file.")
        return {
            'file_name': file_name,
            'status': 'skipped',
            'reason': 'no_cancelled_flights'
        }

    cancellation_rate = cancelled_count / total_count * 100
    print(f"Overall cancellation rate: {cancelled_count}/{total_count} ({cancellation_rate:.2f}%)")

    # Analyze cancellation rates by red-eye status
    redeye_df = flight_df[flight_df['IS_REDEYE'] == 1]
    non_redeye_df = flight_df[flight_df['IS_REDEYE'] == 0]

    if len(redeye_df) > 0:
        redeye_cancel_rate = redeye_df['IS_CANCELLED'].mean() * 100
        print(f"Red-eye flights cancellation rate: {redeye_cancel_rate:.2f}%")

    if len(non_redeye_df) > 0:
        non_redeye_cancel_rate = non_redeye_df['IS_CANCELLED'].mean() * 100
        print(f"Non-red-eye flights cancellation rate: {non_redeye_cancel_rate:.2f}%")

    # Analyze cancellation rates by weekend status
    weekend_df = flight_df[flight_df['IS_WEEKEND'] == 1]
    weekday_df = flight_df[flight_df['IS_WEEKEND'] == 0]

    if len(weekend_df) > 0:
        weekend_cancel_rate = weekend_df['IS_CANCELLED'].mean() * 100
        print(f"Weekend flights cancellation rate: {weekend_cancel_rate:.2f}%")

    if len(weekday_df) > 0:
        weekday_cancel_rate = weekday_df['IS_CANCELLED'].mean() * 100
        print(f"Weekday flights cancellation rate: {weekday_cancel_rate:.2f}%")

    # Analyze cancellation rates by peak times
    morning_peak_df = flight_df[flight_df['IS_MORNING_PEAK'] == 1]
    evening_peak_df = flight_df[flight_df['IS_EVENING_PEAK'] == 1]
    off_peak_df = flight_df[(flight_df['IS_MORNING_PEAK'] == 0) & (flight_df['IS_EVENING_PEAK'] == 0)]

    if len(morning_peak_df) > 0:
        morning_cancel_rate = morning_peak_df['IS_CANCELLED'].mean() * 100
        print(f"Morning peak flights cancellation rate: {morning_cancel_rate:.2f}%")

    if len(evening_peak_df) > 0:
        evening_cancel_rate = evening_peak_df['IS_CANCELLED'].mean() * 100
        print(f"Evening peak flights cancellation rate: {evening_cancel_rate:.2f}%")

    if len(off_peak_df) > 0:
        off_peak_cancel_rate = off_peak_df['IS_CANCELLED'].mean() * 100
        print(f"Off-peak flights cancellation rate: {off_peak_cancel_rate:.2f}%")

    print("Matching weather data with flights...")
    flight_df = match_weather_data(flight_df)
    flight_df = match_destination_weather_data(flight_df)  # NEW: Add destination weather

    # Feature selection - Now including destination weather features
    print("Selecting features...")

    cat_features = ["YEAR", "WEEK", 'MKT_AIRLINE', 'ORIGIN_IATA', 'DEST_IATA', 'IS_REDEYE',
                     'IS_WEEKEND', 'IS_MORNING_PEAK', 'IS_EVENING_PEAK', 'EXTREME_WEATHER', 'DEST_EXTREME_WEATHER']
    num_features = ['DISTANCE', 'PRCP', 'DEST_PRCP']

    # Ensure all selected features exist in the dataframe
    cat_features = [f for f in cat_features if f in flight_df.columns]
    num_features = [f for f in num_features if f in flight_df.columns]

    if not cat_features or not num_features:
        print("Missing required features. Skipping file.")
        return {
            'file_name': file_name,
            'status': 'skipped',
            'reason': 'missing_required_features'
        }

    print(f"Using categorical features: {cat_features}")
    print(f"Using numerical features: {num_features}")

    X = flight_df[cat_features + num_features].copy()
    y = flight_df['IS_CANCELLED'].copy()

    for col in cat_features:
        if X[col].isnull().sum() > 0:
            X[col].fillna('unknown', inplace=True)
    for col in num_features:
        if X[col].isnull().sum() > 0:
            X[col].fillna(X[col].median(), inplace=True)

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2025, stratify=y)
    print(f"Training set size: {X_train.shape}")
    print(f"Test set size: {X_test.shape}")

    # Train model
    print("Training Random Forest model...")
    model_start_time = time.time()

    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, num_features),
            ('cat', categorical_transformer, cat_features)
        ])

    # Create and train Random Forest model
    model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(
            n_estimators=250,
            max_depth=8,
            min_samples_split=20,
            min_samples_leaf=10,
            class_weight={0: 1, 1: 5},  # Handle class imbalance
            max_features='sqrt',
            max_samples=0.9,        # Use 90% of samples per tree
            random_state=2025,
            n_jobs=-1
        ))
    ])

    # Train the model
    model.fit(X_train, y_train)
    final_model = model

    # Predictions
    y_pred = final_model.predict(X_test)
    y_prob = final_model.predict_proba(X_test)[:, 1]

    # Feature importance from Random Forest model
    try:
        feature_names = final_model.named_steps['preprocessor'].get_feature_names_out()
        rf_model = final_model.named_steps['classifier']
        importances = rf_model.feature_importances_

        feature_importance = pd.DataFrame({
            'Feature': feature_names,
            'Importance': importances
        })

        feature_importance = feature_importance.sort_values('Importance', ascending=False)

    except Exception as e:
        print(f"Error extracting feature importances: {e}")
        feature_importance = pd.DataFrame()

    model_training_time = time.time() - model_start_time
    print(f"Model training took: {model_training_time:.2f} seconds")

    # Evaluate model
    print("Evaluating model...")

    if not isinstance(y_pred, np.ndarray) or len(y_pred) == 0:
        print("No predictions available. Skipping evaluation.")
        return {
            'file_name': file_name,
            'status': 'error',
            'reason': 'prediction_failed'
        }

    # Calculate metrics
    try:
        accuracy = (y_pred == y_test).mean() * 100
        roc_auc = roc_auc_score(y_test, y_prob)

        report = classification_report(y_test, y_pred, output_dict=True)

        cm = confusion_matrix(y_test, y_pred)

        print(f"Accuracy: {accuracy:.2f}%")
        print(f"ROC AUC: {roc_auc:.4f}")

        if not feature_importance.empty:
            print("\nTop 10 most important features:")
            print(feature_importance.head(10))

            print("\nImportance of indicator variables:")

            for indicator in ['IS_REDEYE', 'IS_WEEKEND', 'IS_MORNING_PEAK', 'IS_EVENING_PEAK']:
                indicator_features = [f for f in feature_importance['Feature'].tolist() if indicator in f]

                if indicator_features:
                    for feat in indicator_features:
                        importance = feature_importance[feature_importance['Feature'] == feat].iloc[0]['Importance']
                        rank = feature_importance[feature_importance['Feature'] == feat].index[0] + 1
                        print(f"{feat} importance: {importance:.6f} (rank: {rank} out of {len(feature_importance)})")

            print("\nImportance of weather features:")
            weather_features = [f for f in feature_importance['Feature'].tolist()
                                if 'PRCP' in f or 'EXTREME_WEATHER' in f]

            if weather_features:
                for feat in weather_features:
                    importance = feature_importance[feature_importance['Feature'] == feat].iloc[0]['Importance']
                    rank = feature_importance[feature_importance['Feature'] == feat].index[0] + 1
                    print(f"{feat} importance: {importance:.6f} (rank: {rank} out of {len(feature_importance)})")

            # Save feature importance to CSV
            feature_importance.to_csv(os.path.join(output_dir, 'metrics', f"{model_name}_feature_importance.csv"), index=False)

            # Plot feature importance
            plt.figure(figsize=(16, 10))
            top_features = feature_importance.head(15)  # Top 15 features
            sns.barplot(x='Importance', y='Feature', data=top_features)
            plt.title(f'Top 15 Feature Importances for {model_name}')
            plt.tight_layout()
            plt.savefig(os.path.join(output_dir, 'plots', f"{model_name}_feature_importance.png"))
            plt.close()
        else:
            print("Could not extract feature importances")

        # Plot ROC curve
        plt.figure(figsize=(16, 10))
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.4f})')
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'ROC Curve for {model_name} (Random Forest)')
        plt.legend()
        plt.savefig(os.path.join(output_dir, 'plots', f"{model_name}_roc_curve.png"))
        plt.close()

        # Plot confusion matrix
        plt.figure(figsize=(16, 10))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                   xticklabels=['Not Cancelled', 'Cancelled'],
                   yticklabels=['Not Cancelled', 'Cancelled'])
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.title(f'Confusion Matrix for {model_name}')
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'plots', f"{model_name}_confusion_matrix.png"))
        plt.close()

        # Plot cancellation rate comparison for all indicators
        plt.figure(figsize=(16, 12))

        # Set up the categories and their corresponding rates
        categories = []
        rates = []
        counts = []
        colors = []

        # Add red-eye vs non-red-eye
        if len(redeye_df) > 0 and len(non_redeye_df) > 0:
            categories.extend(['Non-Red-Eye', 'Red-Eye'])
            rates.extend([non_redeye_cancel_rate, redeye_cancel_rate])
            counts.extend([len(non_redeye_df), len(redeye_df)])
            colors.extend(['skyblue', 'navy'])

        # Add weekend vs weekday
        if len(weekend_df) > 0 and len(weekday_df) > 0:
            categories.extend(['Weekday', 'Weekend'])
            rates.extend([weekday_cancel_rate, weekend_cancel_rate])
            counts.extend([len(weekday_df), len(weekend_df)])
            colors.extend(['lightgreen', 'darkgreen'])

        # Add peak times
        if len(morning_peak_df) > 0 and len(evening_peak_df) > 0 and len(off_peak_df) > 0:
            categories.extend(['Off-Peak', 'Morning Peak', 'Evening Peak'])
            rates.extend([off_peak_cancel_rate, morning_cancel_rate, evening_cancel_rate])
            counts.extend([len(off_peak_df), len(morning_peak_df), len(evening_peak_df)])
            colors.extend(['lightcoral', 'salmon', 'firebrick'])

        # Create the bar chart
        plt.figure(figsize=(18, 10))
        bars = plt.bar(categories, rates, color=colors)

        # Add value labels on top of bars
        for i, (bar, rate, count) in enumerate(zip(bars, rates, counts)):
            plt.text(i, rate + 0.5, f"{rate:.2f}%\n({count} flights)",
                     ha='center', va='bottom')

        plt.ylabel('Cancellation Rate (%)')
        plt.title(f'Cancellation Rate Comparison ({model_name})')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'plots', f"{model_name}_indicator_comparison.png"))
        plt.close()

        # Generate partial dependence plots for indicators
        try:
            indicator_effects = {}
            for indicator in ['IS_REDEYE', 'IS_WEEKEND', 'IS_MORNING_PEAK', 'IS_EVENING_PEAK']:
                if indicator in X_test.columns:

                    positive_X_test = X_test.copy()
                    positive_X_test[indicator] = 1

                    negative_X_test = X_test.copy()
                    negative_X_test[indicator] = 0

                    positive_probs = final_model.predict_proba(positive_X_test)[:, 1]
                    negative_probs = final_model.predict_proba(negative_X_test)[:, 1]

                    avg_effect = np.mean(positive_probs - negative_probs)

                    # Store the effect
                    indicator_effects[indicator] = {
                        'avg_effect': avg_effect,
                        'differences': positive_probs - negative_probs
                    }

                    # Plot distribution of effects for this indicator
                    plt.figure(figsize=(16, 10))

                    # Plot the differences for each sample
                    differences = positive_probs - negative_probs
                    sns.histplot(differences, bins=30, kde=True)

                    plt.axvline(avg_effect, color='red', linestyle='--',
                               label=f'Average effect: {avg_effect:.4f}')
                    plt.xlabel(f'Change in Cancellation Probability ({indicator}=1 vs {indicator}=0)')
                    plt.ylabel('Frequency')
                    plt.title(f'Effect of {indicator} on Cancellation Probability ({model_name})')
                    plt.legend()
                    plt.tight_layout()
                    plt.savefig(os.path.join(output_dir, 'plots', f"{model_name}_{indicator}_effect.png"))
                    plt.close()

                    print(f"Average effect of {indicator}=1 on cancellation probability: {avg_effect:.4f}")

            # Create a comparison of all indicator effects
            if indicator_effects:
                plt.figure(figsize=(16, 10))
                indicators = list(indicator_effects.keys())
                effects = [indicator_effects[ind]['avg_effect'] for ind in indicators]

                colors = ['red' if e > 0 else 'green' for e in effects]

                bars = plt.bar(indicators, effects, color=colors)

                # Add value labels
                for i, (bar, effect) in enumerate(zip(bars, effects)):
                    plt.text(i, effect + 0.001 if effect > 0 else effect - 0.003,
                             f"{effect:.4f}", ha='center', va='center' if effect > 0 else 'top')

                plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
                plt.ylabel('Average Effect on Cancellation Probability')
                plt.title(f'Comparison of Indicator Effects on Cancellation Probability ({model_name})')
                plt.tight_layout()
                plt.savefig(os.path.join(output_dir, 'plots', f"{model_name}_all_indicators_effect.png"))
                plt.close()

        except Exception as e:
            print(f"Error creating indicator effect plots: {e}")

        # Save model
        model_path = os.path.join(output_dir, f"{model_name}_model.joblib")
        dump(final_model, model_path)
        print(f"Model saved to {model_path}")

        # Save metrics summary
        metrics = {
            'file_name': file_name,
            'model_name': model_name,
            'file_year': file_year,
            'accuracy': accuracy,
            'roc_auc': roc_auc,
            'precision': report['1']['precision'],
            'recall': report['1']['recall'],
            'f1_score': report['1']['f1-score'],
            'cancellation_rate': cancellation_rate,
            'training_time': model_training_time,
            'training_size': len(X_train),
            'test_size': len(X_test),
            'status': 'success'
        }

        # Add indicator-specific metrics
        metrics['redeye_count'] = len(redeye_df)
        metrics['redeye_percentage'] = len(redeye_df) / len(flight_df) * 100
        metrics['redeye_cancel_rate'] = redeye_cancel_rate if len(redeye_df) > 0 else None
        metrics['non_redeye_cancel_rate'] = non_redeye_cancel_rate if len(non_redeye_df) > 0 else None

        # Weekend metrics
        metrics['weekend_count'] = len(weekend_df)
        metrics['weekend_percentage'] = len(weekend_df) / len(flight_df) * 100
        metrics['weekend_cancel_rate'] = weekend_cancel_rate if len(weekend_df) > 0 else None
        metrics['weekday_cancel_rate'] = weekday_cancel_rate if len(weekday_df) > 0 else None

        # Peak time metrics
        metrics['morning_peak_count'] = len(morning_peak_df)
        metrics['morning_peak_percentage'] = len(morning_peak_df) / len(flight_df) * 100
        metrics['morning_peak_cancel_rate'] = morning_cancel_rate if len(morning_peak_df) > 0 else None

        metrics['evening_peak_count'] = len(evening_peak_df)
        metrics['evening_peak_percentage'] = len(evening_peak_df) / len(flight_df) * 100
        metrics['evening_peak_cancel_rate'] = evening_cancel_rate if len(evening_peak_df) > 0 else None

        metrics['off_peak_count'] = len(off_peak_df)
        metrics['off_peak_percentage'] = len(off_peak_df) / len(flight_df) * 100
        metrics['off_peak_cancel_rate'] = off_peak_cancel_rate if len(off_peak_df) > 0 else None

        # Feature importance for indicators
        for indicator in ['IS_REDEYE', 'IS_WEEKEND', 'IS_MORNING_PEAK', 'IS_EVENING_PEAK']:
            indicator_features = [f for f in feature_importance['Feature'].tolist() if indicator in f]
            if indicator_features:
                for feat in indicator_features:
                    importance = feature_importance[feature_importance['Feature'] == feat].iloc[0]['Importance']
                    metrics[f'{indicator.lower()}_importance'] = importance

        # Feature importance for weather features - now including destination weather
        weather_features = [f for f in feature_importance['Feature'].tolist()
                           if 'PRCP' in f or 'EXTREME_WEATHER' in f]
        if weather_features:
            for feat in weather_features:
                importance = feature_importance[feature_importance['Feature'] == feat].iloc[0]['Importance']
                feat_key = feat.replace('cat__', '').replace('num__', '')
                metrics[f'{feat_key}_importance'] = importance

        # Add indicator effect from partial dependence if available
        if 'indicator_effects' in locals():
            for indicator, effect_data in indicator_effects.items():
                metrics[f'{indicator.lower()}_effect'] = effect_data['avg_effect']

        # Save confusion matrix values
        metrics['true_negative'] = cm[0, 0]
        metrics['false_positive'] = cm[0, 1]
        metrics['false_negative'] = cm[1, 0]
        metrics['true_positive'] = cm[1, 1]

        # Save top 5 most important features
        if not feature_importance.empty:
            for i in range(min(5, len(feature_importance))):
                feat = feature_importance.iloc[i]
                metrics[f'top_feature_{i+1}'] = feat['Feature']
                metrics[f'top_feature_{i+1}_importance'] = feat['Importance']

        # Add weather data match rates
        origin_matched = flight_df['PRCP'].notnull().sum()
        dest_matched = flight_df['DEST_PRCP'].notnull().sum() if 'DEST_PRCP' in flight_df.columns else 0

        metrics['origin_weather_match_rate'] = origin_matched / len(flight_df) * 100
        metrics['dest_weather_match_rate'] = dest_matched / len(flight_df) * 100

        print(f"Processing of {file_name} completed in {time.time() - start_time:.2f} seconds")
        return metrics

    except Exception as e:
        print(f"Error in evaluation: {e}")
        return {
            'file_name': file_name,
            'status': 'error',
            'reason': str(e)
        }

In [63]:

# Sequential processing of the May files
results = []
for i, file_path in enumerate(flight_files):
    result = train_model_for_file(file_path, i, len(flight_files))
    results.append(result)

# Summarize results
print("\nSummary of Random Forest model training:")
success_count = sum(1 for r in results if r.get('status') == 'success')
error_count = sum(1 for r in results if r.get('status') == 'error')
skipped_count = sum(1 for r in results if r.get('status') == 'skipped')

print(f"Successfully trained models: {success_count}/{len(results)}")
print(f"Failed models: {error_count}/{len(results)}")
print(f"Skipped files: {skipped_count}/{len(results)}")

results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join(output_dir, 'cancelled_prob_rf_summary.csv'), index=False)

# Calculate average metrics
if success_count > 0:
    successful_results = [r for r in results if r.get('status') == 'success']
    avg_accuracy = sum(r.get('accuracy', 0) for r in successful_results) / success_count
    avg_roc_auc = sum(r.get('roc_auc', 0) for r in successful_results) / success_count
    avg_precision = sum(r.get('precision', 0) for r in successful_results) / success_count
    avg_recall = sum(r.get('recall', 0) for r in successful_results) / success_count
    avg_origin_weather_match = sum(r.get('origin_weather_match_rate', 0) for r in successful_results) / success_count if any('origin_weather_match_rate' in r for r in successful_results) else 0
    avg_dest_weather_match = sum(r.get('dest_weather_match_rate', 0) for r in successful_results) / success_count if any('dest_weather_match_rate' in r for r in successful_results) else 0

    print("\nAverage metrics across all successful models:")
    print(f"Accuracy: {avg_accuracy:.2f}%")
    print(f"ROC AUC: {avg_roc_auc:.4f}")
    print(f"Precision: {avg_precision:.4f}")
    print(f"Recall: {avg_recall:.4f}")
    print(f"Origin weather data match rate: {avg_origin_weather_match:.2f}%")
    print(f"Destination weather data match rate: {avg_dest_weather_match:.2f}%")

    # For successful models, identify most common important features
    feature_counts = {}
    for result in successful_results:
        for i in range(1, 6):  # Top 5 features
            feature_key = f'top_feature_{i}'
            if feature_key in result:
                feature = result[feature_key]
                if feature in feature_counts:
                    feature_counts[feature] += 1
                else:
                    feature_counts[feature] = 1

    if feature_counts:
        print("\nMost common important features across all models:")
        sorted_features = sorted(feature_counts.items(), key=lambda x: x[1], reverse=True)
        for feature, count in sorted_features[:10]:  # Top 10 most common
            print(f"{feature}: Appears in {count} models ({count/success_count*100:.1f}%)")

    # Analyze all indicator effects across all models
    indicators = ['is_redeye', 'is_weekend', 'is_morning_peak', 'is_evening_peak']
    for indicator in indicators:
        effect_key = f'{indicator}_effect'
        importance_key = f'{indicator}_importance'

        if all(effect_key in r for r in successful_results):
            effects = [r[effect_key] for r in successful_results]
            avg_effect = sum(effects) / len(effects)

            effect_dir = "increases" if avg_effect > 0 else "decreases"
            print(f"\n{indicator.upper()} generally {effect_dir} cancellation probability by {abs(avg_effect):.4f}")

        if all(importance_key in r for r in successful_results):
            importances = [r[importance_key] for r in successful_results]
            avg_importance = sum(importances) / len(importances)

            print(f"Average {indicator.upper()} importance: {avg_importance:.6f}")

    # Analyze all weather feature importance across all models
    weather_indicators = ['prcp', 'extreme_weather', 'dest_prcp', 'dest_extreme_weather']
    for indicator in weather_indicators:
        importance_key = f'{indicator}_importance'

        if any(importance_key in r for r in successful_results):
            results_with_feature = [r for r in successful_results if importance_key in r]
            if results_with_feature:
                importances = [r[importance_key] for r in results_with_feature]
                avg_importance = sum(importances) / len(importances)

                print(f"Average {indicator.upper()} importance: {avg_importance:.6f} (found in {len(results_with_feature)}/{success_count} models)")

    # Plot combined indicator effects across all models
    all_indicators = []
    all_effects = []

    for indicator in indicators:
        effect_key = f'{indicator}_effect'
        if all(effect_key in r for r in successful_results):
            all_indicators.append(indicator.upper())
            effects = [r[effect_key] for r in successful_results]
            avg_effect = sum(effects) / len(effects)
            all_effects.append(avg_effect)

    if all_indicators:
        plt.figure(figsize=(16, 10))

        # Determine colors based on effect direction
        colors = ['red' if e > 0 else 'green' for e in all_effects]

        bars = plt.bar(all_indicators, all_effects, color=colors)

        # Add value labels
        for i, (bar, effect) in enumerate(zip(bars, all_effects)):
            plt.text(i, effect + 0.001 if effect > 0 else effect - 0.003,
                     f"{effect:.4f}", ha='center', va='center' if effect > 0 else 'top')

        plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
        plt.ylabel('Average Effect on Cancellation Probability')
        plt.title('Average Indicator Effects Across All Models')
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'plots', 'average_all_indicators_effect.png'))
        plt.close()
        print("\nAverage indicator effects comparison saved to average_all_indicators_effect.png")

print("\nRandom Forest model training with enhanced indicators and destination weather complete!")
print(f"Full summary saved to {os.path.join(output_dir, 'cancelled_prob_rf_summary.csv')}")


Processing file 1/4: May2021.csv (May 2021)
Months found in data: {5: 520059}
Filtered to only May data: 520059 rows
Filtered from 520059 to 171867 rows for top 30 airports
Standardizing day of week...
Day of week range: 0 to 6 (0=Sunday, 1=Monday, ..., 6=Saturday)
Creating red-eye flight indicator...
Identified 7202 red-eye flights out of 171867 total flights (4.19%)
Creating additional indicators: IS_WEEKEND, IS_MORNING_PEAK, IS_EVENING_PEAK...
Identified 54383 weekend flights (31.64%)
Identified 37460 morning peak flights (21.80%)
Identified 30873 evening peak flights (17.96%)
Final dataset shape: (171867, 55)
Preprocessing data...
Overall cancellation rate: 485/171867 (0.28%)
Red-eye flights cancellation rate: 0.22%
Non-red-eye flights cancellation rate: 0.28%
Weekend flights cancellation rate: 0.15%
Weekday flights cancellation rate: 0.34%
Morning peak flights cancellation rate: 0.25%
Evening peak flights cancellation rate: 0.35%
Off-peak flights cancellation rate: 0.27%
Matching

<Figure size 1600x1200 with 0 Axes>

<Figure size 1600x1200 with 0 Axes>

<Figure size 1600x1200 with 0 Axes>

<Figure size 1600x1200 with 0 Axes>